In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np

Here, we define the columns of the training set, and load it from the file.

In [23]:
trainingSetStamp='1561827739132'
training_column_names=["HAND_TYPE",
              "FINGER1_X", "FINGER1_Y", "FINGER1_Z",
              "FINGER2_X", "FINGER2_Y", "FINGER2_Z",
              "FINGER3_X", "FINGER3_Y", "FINGER3_Z",
              "FINGER4_X", "FINGER4_Y", "FINGER4_Z",
              "FINGER5_X", "FINGER5_Y", "FINGER5_Z"]
column_names=training_column_names[:]
column_names.append("INSTRUCTION_LABEL")
data = pd.read_csv('../../trainingAssets/sets/trainingSet_'+trainingSetStamp+'.txt', sep="|", header=None, names=column_names)

data

,HAND_TYPE,FINGER1_X,FINGER1_Y,FINGER1_Z,FINGER2_X,FINGER2_Y,FINGER2_Z,FINGER3_X,FINGER3_Y,FINGER3_Z,FINGER4_X,FINGER4_Y,FINGER4_Z,FINGER5_X,FINGER5_Y,FINGER5_Z,INSTRUCTION_LABEL
0,0,1.39083,144.674,74.740900,-10.1887,141.819,-39.49540,-46.7525,140.227,-69.73950,-84.6680,136.601,-72.44210,-141.9800,122.710,-50.775100,0
1,0,1.39103,144.669,74.754600,-10.1981,141.806,-39.48650,-46.7611,140.226,-69.73230,-84.6704,136.600,-72.43790,-141.9860,122.712,-50.767900,0
2,0,1.39263,144.666,74.778400,-10.2212,141.766,-39.47080,-46.7810,140.222,-69.71860,-84.6740,136.598,-72.43110,-141.9940,122.714,-50.755000,0
3,0,1.39638,144.670,74.801200,-10.2543,141.705,-39.46670,-46.8092,140.216,-69.70450,-84.6769,136.595,-72.42410,-142.0020,122.718,-50.740600,0
4,0,1.40073,144.675,74.821200,-10.2960,141.634,-39.49050,-46.8449,140.208,-69.69510,-84.6790,136.593,-72.41870,-142.0080,122.725,-50.726800,0
5,0,1.40646,144.680,74.838100,-10.3293,141.576,-39.55320,-46.8841,140.196,-69.70070,-84.6796,136.592,-72.41560,-142.0110,122.734,-50.715900,0
6,0,1.41320,144.684,74.853900,-10.3488,141.533,-39.62350,-46.9180,140.184,-69.71760,-84.6788,136.590,-72.41350,-142.0110,122.747,-50.706100,0
7,0,1.42004,144.690,74.870100,-10.3598,141.500,-39.68270,-46.9421,140.173,-69.73600,-84.6760,136.588,-72.41120,-142.0060,122.766,-50.696500,0
8,0,1.42843,144.698,74.887200,-10.3644,141.475,-39.73470,-46.9564,140.164,-69.75520,-84.6701,136.584,-72.41000,-141.9960,122.792,-50.689400,0
9,0,1.44186,144.708,74.909500,-10.3641,141.455,-39.78200,-46.9633,140.156,-69.77580,-84.6601,136.579,-72.41170,-141.9780,122.821,-50.689700,0


Now, we shuffle the data and partition it into training set and test set

In [24]:
data = data.sample(frac=1).reset_index(drop=True)
test_len = (len(data) * 20)//100;
training_data = data[test_len:]
test_data = data[:test_len]

Here we define the feature columns, describing the data on the data frame. 

TODO: Fix the type: The columns HAND_TYPE and INSTRUCTION_LABEL are int, not float32

In [25]:
feature_columns=[tf.contrib.layers.real_valued_column(column_name, dimension=1, dtype=tf.float32) for column_name in training_column_names]

Format the data as required by Tensorflow

In [26]:
x_train_data = { column_name : np.array(training_data[column_name]) for column_name in training_column_names }

Define and train the model: A neural network classifier, with 5 layers with 5 neurons each, and a possible output of 5 labels.

In [27]:
classifier = tf.estimator.DNNClassifier(
       feature_columns = feature_columns,
       hidden_units = [5, 5],
       n_classes = 5)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = x_train_data,
    y = np.array(training_data["INSTRUCTION_LABEL"]).astype(int),
    num_epochs = None,
    shuffle = True)

# Train model.
classifier.train(input_fn = train_input_fn, steps = 2000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpkulb2uvb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f515cce5550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

Use the previously separated test data to evaluate the accuracy of the model

In [28]:
x_testing_data = { column_name : np.array(test_data[column_name]) for column_name in training_column_names }

# Define the training inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = x_testing_data,
    y = np.array(test_data["INSTRUCTION_LABEL"]).astype(int),
    num_epochs = 1,
    shuffle = False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("Test Accuracy: ", accuracy_score)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-29T17:03:55Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpkulb2uvb/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-29-17:03:55
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9410548, average_loss = 0.15197514, global_step = 2000, loss = 18.369995
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmpkulb2uvb/model.ckpt-2000
Test Accuracy:  0.9410548


Export and store the trained model

In [29]:
def serving_input_receiver_fn():
    serialized_tf_example = tf.placeholder(dtype = tf.string, shape = [None], name = 'input_tensors')
    receiver_tensors      = {'predictor_inputs' : serialized_tf_example}
    feature_spec          = { column_name : tf.FixedLenFeature([25], tf.float32) for column_name in training_column_names }
    features              = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)


model_dir = classifier.export_savedmodel(export_dir_base = "../../trainingAssets/models/"+trainingSetStamp, 
                             serving_input_receiver_fn = serving_input_receiver_fn,
                             as_text = True)
print('Model exported to '+ model_dir.decode())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/tmpkulb2uvb/model.ckpt-2000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../../trainingAssets/models/1561827739132/temp-b'1561827840'/saved_model.pbtxt
Model exported to ../../trainingAssets/models/1561827739132/1561827840
